# Create VIEWs on SEC DERA data by LEI

## Full-year FY REVENUES

## Point-in-time FLOAT, DEBT, CASH, ASSETS, (EV = FLOAT + DEBT - CASH, EVIC = FLOAT + DEBT)

In [ ]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [ ]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

import pandas as pd

In [ ]:
from osc_ingest_trino import *

trino_bucket = attach_s3_bucket("S3_DEV")

In [ ]:
for tbl in ['t_r', 't_f', 't_d', 't_c', 't_a']:
    qres = engine.execute(f"drop table if exists sec_dera.{tbl}")
    print(qres.fetchall())

### Revenue

In [ ]:
qres = engine.execute("drop table if exists sec_dera.t_r").fetchall()
qres = engine.execute("""
create or replace view sec_dera.fy_revenue_by_lei as
select name, lei,
       -- tname,
       fy, N.ddate, sic, max(value) as revenue, S.adsh, cik
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate and S.fp='FY'
where (S.form='10-K' or S.form='20-F')
and S.fy>=DATE('2019-01-01')
and coreg is NULL
and qtrs=4
and uom='USD'
and (N.tag='Revenues'
     or N.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
     or N.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
     or N.tag='RevenuesNetOfInterestExpense'
     or N.tag='RegulatedAndUnregulatedOperatingRevenue'
     or N.tag='RegulatedOperatingRevenuePipelines')
group by name, lei,
      fy, N.ddate, sic, S.adsh, cik
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_r as
select * from sec_dera.fy_revenue_by_lei order by fy asc, LEI desc
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.t_r
""")
display(qres.fetchall())

In [ ]:
engine.execute("select * from sec_dera.t_r where name='INDEPENDENT BANK CORP'").fetchall()

### Float (market cap)

In [ ]:
qres = engine.execute("drop table if exists sec_dera.t_f").fetchall()
qres = engine.execute("""
create or replace view sec_dera.float_by_adsh_ddate as
select S.adsh, fy, max(ddate) as f_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate and S.fp='FY'
where (S.form='10-K' or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and (tag='EntityPublicFloat'
             or tag='EntitysPublicFloat'
             or tag='FreeFloat'
             or tag='PublicFloat'
             or tag='PublicFloatValue')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.float_by_lei as
select name, lei, tname, S.fy, N.ddate, sic, max(value) as market_cap, S.adsh
from sec_dera.sub as S
     join sec_dera.float_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.f_ddate=N.ddate
     left join sec_dera.ticker T on S.cik=T.cik
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and value>0
and coreg is NULL
and qtrs=0
and uom='USD'
and (tag='EntityPublicFloat'
     or tag='EntitysPublicFloat'
     or tag='FreeFloat'
     or tag='PublicFloat'
     or tag='PublicFloatValue')
group by name, lei, tname, S.fy, N.ddate, sic, S.adsh
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_f as select * from sec_dera.float_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.t_f
""")
display(qres.fetchall())

In [ ]:
engine.execute("select * from sec_dera.t_f where name='INDEPENDENT BANK CORP'").fetchall()

### Cash

In [ ]:
qres = engine.execute("drop table if exists sec_dera.t_c").fetchall()
qres = engine.execute("""
create or replace view sec_dera.cash_by_adsh_ddate as
select S.adsh, fy, max(ddate) as c_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate and S.fp='FY'
where (S.form='10-K' or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and (tag='Cash'
             or tag='CashAndDueFromBanks'
             or tag='CashAndCashEquivalents'
             or lower(tag)=lower('CashAndCashEquivalentsUnrestricted')
             or tag='CashEquivalentsAtCarryingValue' 
             or tag='CashAndCashEquivalentsAtCarryingValue'
             or tag='CashAndCashEquivalentsAtCarryingValueExcludingVariableInterestEntities')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.cash_by_lei as
select name, lei, S.fy, N.ddate, sic, max(value) as cash
from sec_dera.sub as S
     join sec_dera.cash_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.c_ddate=N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and coreg is NULL
and qtrs=0
and uom='USD'
and (tag='CashAndCashEquivalentsAtCarryingValue' or tag='Cash' or tag='CashEquivalentsAtCarryingValue' or tag='CashAndCashEquivalents')
group by name, lei, S.fy, N.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_c as select * from sec_dera.cash_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.t_c
""")
display(qres.fetchall())

In [ ]:
engine.execute("select * from sec_dera.t_c where name='INDEPENDENT BANK CORP'").fetchall()

### Debt

In [ ]:
# Reiterating comment embedded below: we should sum 'LongTermDebtNoncurrent' and 'LongTermDebtCurrent' as one term we MAX with others
qres = engine.execute("drop table if exists sec_dera.t_d").fetchall()
qres = engine.execute("""
create or replace view sec_dera.debt_by_adsh_ddate as
select S.adsh, fy, max(ddate) as d_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate and S.fp='FY'
where (S.form='10-K' or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
             or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
             or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
             or tag='OperatingLeaseLiabilityNoncurrent'
             or tag='SubordinatedDebt' or tag='ConvertibleDebt'
             or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
             -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
             or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.debt_by_lei as
select name, lei, S.fy, N.ddate, sic, max(value) as debt
from sec_dera.sub as S
     join sec_dera.debt_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and coreg is NULL
and qtrs=0
and uom='USD'
and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
     or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
     or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
     or tag='OperatingLeaseLiabilityNoncurrent'
     or tag='SubordinatedDebt' or tag='ConvertibleDebt'
     or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
     -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
     or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by name, lei, S.fy, N.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_d as select * from sec_dera.debt_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.t_d
""")
display(qres.fetchall())

In [ ]:
engine.execute("select * from sec_dera.t_d where name='INDEPENDENT BANK CORP'").fetchall()

### Assets

In [ ]:
qres = engine.execute("drop table if exists sec_dera.t_a").fetchall()
qres = engine.execute("""
create or replace view sec_dera.assets_by_adsh_ddate as
select S.adsh, fy, max(ddate) as a_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate and S.fp='FY'
where (S.form='10-K' or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and tag='Assets'
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.assets_by_lei as
select name, lei, S.fy, N.ddate, sic, value as assets, S.adsh
from sec_dera.sub as S
     join sec_dera.assets_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.a_ddate=N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A'  or S.form='20-F')
and coreg is NULL
and qtrs=0
and uom='USD'
and tag='Assets'
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_a as select * from sec_dera.assets_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.t_a
""")
display(qres.fetchall())

In [ ]:
engine.execute("select * from sec_dera.t_a where name='INDEPENDENT BANK CORP'").fetchall()

In [ ]:
qres = engine.execute("""
create or replace view sec_dera.financials_by_lei as
select R.name, R.lei,
       tname,
       R.fy, R.ddate, R.sic, revenue, cash, debt, assets, market_cap
       , C.ddate as c_d, D.ddate as d_d, A.ddate as a_d, F.ddate as f_d
from sec_dera.t_r as R
     left join sec_dera.t_c as C on R.name=C.name and R.fy=C.fy
     left join sec_dera.t_d as D on R.name=D.name and R.fy=D.fy
     left join sec_dera.t_a as A on R.name=A.name and R.adsh=A.adsh
     left join sec_dera.t_f as F on R.name=F.name and R.adsh=F.adsh
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.financials_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.financials_by_lei where fy>=DATE('2020-01-01') order by LEI limit 100
""")
display(qres.fetchall())

In [ ]:
stop!

In [ ]:
qres = engine.execute("""
explain analyze
select R.name, R.lei, R.tname, R.fy, R.sic, revenue, cash -- , debt --, assets, market_cap
from sec_dera.fy_revenue_by_lei as R
     join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     -- join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     -- join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     -- join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
l = qres.fetchall()

In [ ]:
qres = engine.execute("""
explain analyze
select R.name, R.lei, R.tname, R.fy, R.sic, revenue,
    -- cash,
    debt,
    -- assets,
    market_cap
from sec_dera.fy_revenue_by_lei as R
     -- join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     -- join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
l = qres.fetchall()

In [ ]:
qres = engine.execute("""
create or replace view sec_dera.financials_by_lei as
select name, lei, fy, Nrevenue.ddate as r_ddate, sic,
       max(Nrevenue.value) as revenue,
       max(Ncash.value) as cash,
       max(Ndebt.value) as debt,
       max(Nassets.value) as assets,
       max(Nfloat.value) as market_cap
       -- max(Nfloat.value)+max(Ndebt.value)-max(Ncash.value) as ev,
       -- max(Nfloat.value)+max(Ndebt.value) as evic
from sec_dera.sub as S
      join sec_dera.num as Nrevenue on S.adsh=Nrevenue.adsh
            and Nrevenue.qtrs=4
            and Nrevenue.ddate>=S.fy and date_add('year', 1, S.fy)>Nrevenue.ddate
      join sec_dera.num as Nassets on S.adsh=Nassets.adsh and Nassets.ddate=Nrevenue.ddate
      join sec_dera.num as Ndebt on S.adsh=Ndebt.adsh and Ndebt.ddate=Nrevenue.ddate
      join sec_dera.num as Ncash on S.adsh=Ncash.adsh and Ncash.ddate=Nrevenue.ddate
      join sec_dera.num as Nfloat on S.adsh=Nfloat.adsh
where S.fp='FY' and (S.form='10-K' or S.form='20-F')
      -- and S.lei='I1BZKREC126H0VB1BL91'
and Nrevenue.coreg is NULL
and Nrevenue.uom='USD'
and (Nrevenue.tag='Revenues'
     or Nrevenue.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
     or Nrevenue.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
     or Nrevenue.tag='RevenuesNetOfInterestExpense'
     or Nrevenue.tag='RegulatedAndUnregulatedOperatingRevenue'
     or Nrevenue.tag='RegulatedOperatingRevenuePipelines')
and Nfloat.qtrs=0 and Nfloat.ddate>=fy and date_add('year', 1, fy)>Nfloat.ddate
and Nfloat.coreg is NULL
and Nfloat.uom='USD'
and (Nfloat.tag='EntityPublicFloat'
     or Nfloat.tag='EntitysPublicFloat'
     or Nfloat.tag='FreeFloat'
     or Nfloat.tag='PublicFloat'
     or Nfloat.tag='PublicFloatValue')
and Ncash.qtrs=0
and Ncash.coreg is NULL
and Ncash.uom='USD'
and (Ncash.tag='CashAndCashEquivalentsAtCarryingValue'
     or Ncash.tag='Cash'
     or Ncash.tag='CashEquivalentsAtCarryingValue'
     or Ncash.tag='CashAndCashEquivalents')
and Ndebt.qtrs=0
and Ndebt.coreg is NULL
and Ndebt.uom='USD'
and (Ndebt.tag='LongTermDebt'
     or Ndebt.tag='LongTermDebtFairValue'
     or Ndebt.tag='LongTermDebtAndCapitalLeaseObligations'
     or Ndebt.tag='DebtLongtermAndShorttermCombinedAmount'
     or Ndebt.tag='SecuredDebt'
     or Ndebt.tag='UnsecuredDebt'
     or Ndebt.tag='SubordinatedDebt'
     or Ndebt.tag='ConvertibleDebt')
and Nassets.qtrs=0
and Nassets.coreg is NULL
and Nassets.uom='USD'
and Nassets.tag='Assets'
group by name, lei, fy, Nrevenue.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.financials_by_lei
""")
l = qres.fetchall()

print(len(l))

In [ ]:
columns = [x[0] for x in engine.execute("describe sec_dera.financials_by_lei").fetchall()]

In [ ]:
df = pd.DataFrame(data=l, columns=columns)

In [ ]:
df

In [ ]:
df = df.convert_dtypes()

drop_unmanaged_table("osc_datacommons_dev", "sec_dera", "corp_data_df", engine, trino_bucket, verbose=True)

drop_unmanaged_data("sec_dera", "corp_data_df", trino_bucket, verbose=True)

ingest_unmanaged_parquet(df, "sec_dera", "corp_data_df", trino_bucket, partition_columns=[],
                         append=False, workdir='/tmp', verbose=True)

sql = unmanaged_parquet_tabledef(df, "osc_datacommons_dev", "sec_dera", "corp_data_df", trino_bucket, partition_columns = [],
                                 verbose=True)
qres = engine.execute(sql)
display(qres.fetchall())

qres = engine.execute("select * from sec_dera.corp_data_df")
l = qres.fetchall()

In [ ]:
qres = engine.execute("""
explain select * from sec_dera.financials_by_lei
""")
print(qres.fetchall()[0][0])